In [1]:
# %pip install faster-whisper
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install ffmpeg

In [2]:
import torch

torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
MEDIA_SOURCE = "/var/www/ccai/data-manager/public/audio"

In [12]:
from faster_whisper import WhisperModel
import os
import multiprocessing
import json

model_size = "large-v3"
device = "cuda"

files = [
    file
    for file in os.listdir(MEDIA_SOURCE)
    if os.path.isfile(f"{MEDIA_SOURCE}/{file}")
]


def work(file):
    transcript = []
    model = WhisperModel(model_size, device=device, compute_type="float16")
    segments, info = model.transcribe(
        f"{MEDIA_SOURCE}/{file}",
        beam_size=10,
        without_timestamps=False,
        vad_filter=True,
    )

    transcript.append(f"LANGUAGE: {info.language}")
    transcript.append(f"PROBABILITY: {round(info.language_probability*100)}%")

    for segment in segments:
        transcript.append(
            "[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text)
        )

    # write transcript to file
    print(json.dumps(transcript))
    # t_file = f"{MEDIA_SOURCE}/TRANSCRIPT_{file}.txt"
    # with open(t_file, "a+") as transcript:

    print(f"SUCCESS => Transcript saved at ...")


# Main execution
if __name__ == "__main__":
    # pool = multiprocessing.Pool(processes=2)

    # for file in files:
    #     print(f'START => Transcribing {file}...')
    #     p = multiprocessing.Process(target=work, args=(file,)).start()
    # pool.apply_async(func=work, args=(file,))

    # pool.close()
    # pool.join()

    work("y2mate.com - Sample Order Taking  Customer Support Philippines.mp3")

    torch.cuda.empty_cache()

["LANGUAGE: en", "PROBABILITY: 98%", "[0.47s -> 3.35s]  Thank you for calling Martha's Flowers. How may I assist you?", "[3.79s -> 7.27s]  Hello, I'd like to order flowers and I think you have what I'm looking for.", "[7.73s -> 10.61s]  I'd be happy to take care of your order. May I have your name, please?", "[11.15s -> 11.95s]  Randall Thomas.", "[12.67s -> 14.93s]  Randall Thomas. Can you spell that for me?", "[15.63s -> 22.11s]  Randall, R-A-N-D-A-L-L, Thomas, T-H-O-M-A-S.", "[22.75s -> 27.11s]  Thank you for that information, Randall. May I have your home or office number area code first?", "[27.11s -> 33.67s]  Area code 409, then 866-5088.", "[34.33s -> 40.49s]  That's 409-866-5088. Do you have a fax number or email address?", "[41.21s -> 46.37s]  My email is randall.thomas at gmail.com.", "[46.97s -> 51.49s]  randall.thomas at gmail.com. May I have your shipping address?", "[51.49s -> 57.09s]  6800 Gladys Avenue, Beaumont, Texas.", "[57.11s -> 61.05s]  Zip code is 77706.", "[61.5